In [20]:
with open('keys.txt', 'r') as file:
    # Read the entire contents of the file into a variable
    file_contents = file.read()
    keys = file_contents.split('\n')

import praw

reddit = praw.Reddit(
    client_id=keys[0],
    client_secret=keys[1],
    user_agent=keys[2],
    username=keys[3],
    password=keys[4]
)

In [21]:
subreddit = reddit.subreddit('wallstreetbets')
for post in subreddit.new(limit=100):
    print(post.title)

CVNA Stock Primed for a Short Squeeze
A year salary gone in degen gambling. Ready for the start again
Me and the PLTR regards when the weight of heavy bags are finally lifted.
👀
When will tech join the sell off? 5-30-23 SPY/ ES Futures, QQQ, VIX, 10YR Yield/ DXY Daily Market Analysis
Nvidia's latest 2 hour keynote in 3 minutes
22% of S&P 500 companies mentioned AI on their earnings calls, up from 10% in 2022:
$TSLA puts
2k to 16k $Ai options play
Netflix has finally pulled the trigger on its password-sharing crackdown
Market Recap - 5/20/23 - everything is over bought
What Are Your Moves Tomorrow, May 31, 2023
The odds of a 0.25% interest rate hike in June are now 67%, according to the CME's FedWatch Tool (It also shows that there is now a 30% chan
Ccl calls keep holding waiting for the moon pop.
#inversecathie is the way
NNOX is still bullshit - and now it pumped - big opportunity for REGARDED BEARS
NOKIA
$BUD PUTs +$35k (+127%) - Pour One Out for the “Real Men of Genius”
Palantir Ann

In [22]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob


nlp = spacy.load("en_core_web_sm")
spacy_text_blob = SpacyTextBlob(nlp)
nlp.add_pipe("spacytextblob")

In [23]:
for post in subreddit.new(limit=100):
    doc = nlp(post.title)
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_)

CVNA CVNA PROPN NNP compound
Stock Stock PROPN NNP compound
Primed Primed PROPN NNP ROOT
for for ADP IN prep
a a DET DT det
Short Short PROPN NNP compound
Squeeze Squeeze PROPN NNP pobj
A a DET DT det
year year NOUN NN compound
salary salary NOUN NN nsubj
gone go VERB VBN ROOT
in in ADP IN prep
degen degen NOUN NN compound
gambling gambling NOUN NN pobj
. . PUNCT . punct
Ready ready ADJ JJ ROOT
for for ADP IN prep
the the DET DT det
start start NOUN NN pobj
again again ADV RB advmod
Me I PRON PRP nsubj
and and CCONJ CC cc
the the DET DT det
PLTR PLTR PROPN NNP conj
regards regard VERB VBZ ROOT
when when SCONJ WRB advmod
the the DET DT det
weight weight NOUN NN nsubjpass
of of ADP IN prep
heavy heavy ADJ JJ amod
bags bag NOUN NNS pobj
are be AUX VBP auxpass
finally finally ADV RB advmod
lifted lift VERB VBN advcl
. . PUNCT . punct
👀 👀 PROPN NNP ROOT
When when SCONJ WRB advmod
will will AUX MD aux
tech tech NOUN NN nsubj
join join VERB VB ROOT
the the DET DT det
sell sell NOUN NN dobj
of

In [24]:
def extract_stock_information(text):
    doc = nlp(text)
    entities = []
    sentiments = []

    for entity in doc.ents:
        if entity.label_ == "ORG" and "tsla" in entity.text.lower():
            entities.append(entity.text)

    for sentence in doc.sents:
        sentiment = sentence._.blob.sentiment
        sentiments.append(sentiment)

    return entities, sentiments

In [25]:
for post in subreddit.new(limit=100):
    entities, sentiments = extract_stock_information(post.title)
    #print(post.title + ": " + str(len(sentiments)))
    if sentiments:
        #print("Entities: ", entities)
        print("Sentiments: ", sentiments)


Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.3)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.0), Sentiment(polarity=0.2, subjectivity=0.5)]
Sentiments:  [Sentiment(polarity=-0.1, subjectivity=0.75)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.0)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.0), Sentiment(polarity=0.0, subjectivity=0.0)]
Sentiments:  [Sentiment(polarity=0.5, subjectivity=0.9)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.0)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.0)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.0)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=1.0)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.0)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.0)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.0)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.0)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.0)]
Sentiments:  [Sentiment(polarity=0.0, subjectivity=0.1)]


In [26]:
import yfinance as yf

def retrieve_historical_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

In [27]:
import datetime as datetime

symbol = "TSLA"
start_date = datetime.datetime(2023, 5, 1)
end_date = datetime.datetime(2023, 6, 1)

stock_data = retrieve_historical_stock_data(symbol, start_date, end_date)
type(stock_data.index[0])

[*********************100%***********************]  1 of 1 completed


pandas._libs.tslibs.timestamps.Timestamp

In [28]:
posts = []
for post in subreddit.new(limit = 100):
    # if start_date.timestamp() <= post.created_utc <= end_date.timestamp():
    posts.append({
        'title': post.title,
        'body': post.selftext,
        'date': post.created_utc,
        'score': post.score,
    })
    # elif post.created_utc < start_date.timestamp():
    #     break
import pandas as pd
from datetime import datetime

posts_df = pd.DataFrame(posts)
posts_df['Date'] = pd.to_datetime(posts_df['date'], unit='s').dt.strftime("%Y-%m-%d")
#posts_df = posts_df.drop('date', axis=1)
posts_df['Date'] = pd.to_datetime(posts_df['Date'])
posts_df.head()
posts_df.to_csv("test.csv")



In [29]:
def merge_data(posts, stock_data):
    merged_data = pd.merge(posts, stock_data, how='inner', left_on='Date', right_index=True)
    return merged_data

In [30]:
merged_data = merge_data(posts_df, stock_data)
print(merged_data.dtypes)
merged_data.head()

title                object
body                 object
date                float64
score                 int64
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object


,title,body,date,score,Date,Open,High,Low,Close,Adj Close,Volume
0,CVNA Stock Primed for a Short Squeeze,"\nGreetings, fellow Regards. I've done some DD...",1.685480e+09,0,2023-05-30,200.100006,204.479996,197.529999,201.160004,201.160004,128332764
1,A year salary gone in degen gambling. Ready fo...,,1.685480e+09,6,2023-05-30,200.100006,204.479996,197.529999,201.160004,201.160004,128332764
2,Me and the PLTR regards when the weight of hea...,,1.685480e+09,2,2023-05-30,200.100006,204.479996,197.529999,201.160004,201.160004,128332764
3,👀,Corporate price hikes get the side-eye as #gre...,1.685479e+09,0,2023-05-30,200.100006,204.479996,197.529999,201.160004,201.160004,128332764
4,When will tech join the sell off? 5-30-23 SPY/...,Its actually really interesting the fact that ...,1.685479e+09,5,2023-05-30,200.100006,204.479996,197.529999,201.160004,201.160004,128332764


In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Assume merged_data contains the merged DataFrame with Reddit posts and stock price data
# Ensure the DataFrame has columns for 'date', 'title', 'body', 'score', 'Open', 'Close', etc.

# Prepare the features and target variables

features = merged_data[['score', 'Open', 'Volume', 'date']]  # Adjust the features based on your requirements
target = merged_data['Close']  # Adjust the target variable based on your requirements

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

print(X_train.head())
print(y_train.head())

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

# Print the model performance
print("Train Score:", train_score)
print("Test Score:", test_score)

    score        Open     Volume          date
6      10  200.100006  128332764  1.685478e+09
24     16  200.100006  128332764  1.685459e+09
32      0  200.100006  128332764  1.685451e+09
19     23  200.100006  128332764  1.685464e+09
17    293  200.100006  128332764  1.685469e+09
6     201.160004
24    201.160004
32    201.160004
19    201.160004
17    201.160004
Name: Close, dtype: float64
Train Score: 1.0
Test Score: 1.0
